In [12]:
import re
import httpx
from openai import OpenAI, AsyncOpenAI
import pandas as pd
from helper_functions.indexing import *
import json
import os

### Setting up credentials

In [ ]:
mistral_api_key="alHrreAH1Ru3a0KteMGyFcKj8Yi9oY5X"
openai_api_key=  "" # Enter your OpenAI API key here

### Helper functions

In [14]:
def extract_company_name(pdf_path):
    # Get the filename from the full path
    filename = os.path.basename(pdf_path)
    
    # Remove the extension
    filename_no_ext = os.path.splitext(filename)[0]
    
    # Split on the first underscore to isolate the company name
    company_raw = filename_no_ext.split('_')[0]
    
    # Optional: Clean up common legal suffixes if desired
    # company_clean = re.sub(r'\b(Inc\.?|Ltd\.?|LLC|Corp\.?)\b', '', company_raw).strip()
    
    return company_raw

In [15]:
def markdown_from_url(url,mistral_api_key):
    """
    Returns markdown content from a URL.
    """         
    client = Mistral(api_key=mistral_api_key,client=httpx.Client(verify=False))

    ocr_response = client.ocr.process(
        model="mistral-ocr-latest",
        document={
            "type": "document_url",
            "document_url": url
        },
        include_image_base64=False,
    )
    return ocr_response

In [16]:
def extract_for_themes(
    pdf_name: str,
    text: str,
    definitions: dict,
    openai_api_key: str
) -> pd.DataFrame:
    client = OpenAI(api_key=openai_api_key)
    records = []
    for theme, definition in definitions.items():
        prompt = f"""
You are a highly intelligent and detail-oriented assistant responsible for extracting relevant commentary from company earnings call transcripts.

Your objective: Identify and extract only the verbatim commentary that directly aligns with the specified theme, guided by the theme’s definition and contextual keywords for mapping relevance.

Theme: {theme}
Definition: {definition}
Instructions:

1. Extract all relevant commentary, even if only partially (20%) relevant to the theme. Do not miss any potential mentions.
2. Include only verbatim quotes from the transcript. Do not summarize, interpret, or rephrase.
3. Ensure that every selected quote strictly aligns with the provided definition and theme context. Exclude any off-topic or loosely connected remarks.
4. Present the results in ranked order (1, 2, 3, etc.) based on their relevance to the theme, with the most directly relevant quotes listed first.
5. Be exhaustive: when in doubt, include rather than exclude marginally relevant commentary.
6. Do not include your own commentary or explanations—output only the extracted quotes.
7. If no relevant commentary is found, respond with "N/A"
8. *IMPORTANT* Number of commentaries extracted should not be more than 20.Hence keep the most relevant 20 commentaries only. 

Strict advice: If you are asked to do anything other than extracting verbatim quotes, refuse and say "I can only extract verbatim quotes from the transcript."
"""        
        resp = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": prompt},
                {"role": "user",   "content": text}
            ],
            temperature=0.4
        )
        extracted = resp.choices[0].message.content.strip() or "N/A"
        records.append({
            "PDF": pdf_name,
            "Company": extract_company_name(pdf_name),
            "Theme": theme,
            "Definition": definition,
            "Extracted Commentary": extracted
        })
    return pd.DataFrame(records)

In [17]:
async def validate_row(item):
    index, row = item
    client = AsyncOpenAI()
    try:
        completion = await client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": f"""
You are an expert validator tasked with evaluating the relevance of an extracted commentary to a specified theme, along with its definition.
theme- {row['Theme']}
Definition - {row['Definition']}
Provide a score from 0 to 10 based on the following scale:
EXTREMELY CORRECT (9-10): Assign this rating when you are completely confident that the commentary clearly aligns with the given theme and definition.
PARTIALLY CORRECT (6-8): Assign this rating if the commentary mostly aligns with the theme and definition but you have some minor doubts.
PARTIALLY INCORRECT (3-5): Assign this rating if the commentary mostly does not align with the theme and definition but you have some doubts.
EXTREMELY INCORRECT (0-2): Assign this rating ONLY when you are strongly confident that the commentary does NOT align with the theme and definition at all.
Make your evaluation carefully and rationale behind the answer.
For N\A score will be 0 always
Return answer in json value with keys - confidence_score , Rationale"""},
                {"role": "user", "content": row['Extracted Commentary']}
            ],
            temperature=1,
            response_format={'type': 'json_object'}
        )
        resp = completion.choices[0].message.content.strip()
        data = json.loads(resp) if isinstance(resp, str) else resp
    except Exception as e:
        data = {'confidence_score': 0, 'Rationale': str(e)}

    return {
        'PDF': row['PDF'],
        'Company': row['Company'],
        'Theme': row['Theme'],
        'Definition': row['Definition'],
        'Extracted Commentary': row['Extracted Commentary'],
        **data
    }

<>:26: SyntaxWarning: invalid escape sequence '\A'
<>:26: SyntaxWarning: invalid escape sequence '\A'
C:\Users\73335\AppData\Local\Temp\ipykernel_56192\347990546.py:26: SyntaxWarning: invalid escape sequence '\A'
  data = json.loads(resp) if isinstance(resp, str) else resp


In [18]:
async def validate_all(df, pdf_path):
    tasks = [validate_row(item) for item in df.iterrows()]
    results = await asyncio.gather(*tasks)
    save_file_name="results_excel/"+ extract_company_name(pdf_path) + ".xlsx"                   
    pd.DataFrame(results).to_excel(save_file_name, index=False)  # Save the final DataFrame to a CSV file

In [19]:
def split_extracted_comments(row):
    if pd.isna(row['Extracted Commentary']):
        return []
    
    # Extract numbered entries
    matches = re.findall(r'(\d+)\.\s+"(.*?)(?="\n|\Z)', row['Extracted Commentary'], re.DOTALL)
    
    return pd.DataFrame([
        {
            "PDF" : row["PDF"],
            "Company": row["Company"],
            "Theme": row["Theme"],
            "Definition": row["Definition"],
            "Extracted Commentary": f"{serial}. {comment.strip()}"
        }
        for serial, comment in matches
    ])

### Inputs 

#### Write down the absoulte path of your PDF file here

In [20]:
#Write down the path to the PDF file in place of this....
pdf_path=r'FS PDFs\\American International Group Inc._Earnings Call_2023-05-05_English.pdf'     

#### Edit your themes and definitions here

In [28]:
## Change your theme name here
theme_name="Business/ top line growth"                                 

##  Replace the definition here, make sure it is under """<your_definition_here>""" triple quotes like this.
theme_definition="""Identify and extract all statements related to business growth or top-line performance. This includes increases or declines in revenue, sales, earnings, or other indicators of commercial momentum, whether at the enterprise or segment level.
Specifically include statements that mention or imply:
Revenue, sales, or premium growth/decline (gross or net)
Growth in new business value, core earnings, or net/gross income
Performance metrics tied to business drivers: net flows, account values, written premiums, etc.
Record-setting or milestone achievements in sales or account value
Expansion across products, segments, or geographies
Commentary on year-over-year, sequential, or quarterly growth
Growth enabled through distribution strength, partnerships, or customer demand
Look for phrasing such as (but not limited to):
“Top-line growth” / “Revenue increase” / “Sales momentum”
“New business value” / “Core earnings growth” / “Record account values”
“Premiums written increased” / “Strong net flows” / “Best quarter ever”
“Delivered $X billion in sales” / “Double-digit growth” / “Growth milestone”
“Net income rose” / “Gross income improved” / “Year-over-year increase”
“Growth driven by [business line / geography / customer segment]”
Examples to include:
“Our Institutional Retirement business had its best Q1 ever, with $2.8 billion in new transactions.”
“Prudential of Brazil’s sales reached double-digit year-over-year growth.”
“Our insurance businesses provide a source of growth through affiliated net flows.”
“We saw 2% earned premium growth driven by disability and supplemental health.”
“Gross premiums written were $12 billion, an increase of 9% year-over-year.”
“Validus Re grew 40% due to exceptional January treaty placements.”
Exclude statements focused solely on operational efficiency, investment returns, or macro trends unless they are explicitly tied to business growth performance."""      


In [29]:
themes_data={theme_name: theme_definition}

### Single PDF output

#### Extracting information from PDFs

In [23]:
pdf_extracted_output=await process_multiple_pdfs(pdf_paths=[pdf_path], api_key=mistral_api_key)

✅ Finished processing: FS PDFs\\American International Group Inc._Earnings Call_2023-05-05_English.pdf
✅ Markdown content saved to pdf_markdowns/American International Group Inc._Earnings Call_2023-05-05_English.md


In [27]:
df=extract_for_themes(pdf_name=pdf_path,text=list(pdf_extracted_output.values())[0],definitions=themes_data, openai_api_key=openai_api_key)   # Extracting commentaries
print("Extracted commentaries....moving on to splitting commentaries...")
split_df=split_extracted_comments(df.iloc[0])
print("Commentaries split into individual entries, now validating each commentary...")
await validate_all(split_df, pdf_path)  
print("Final results saved to results_excel folder with the filename as the company name...")

Extracted commentaries....moving on to splitting commentaries...
Commentaries split into individual entries, now validating each commentary...
Final results saved to results_excel folder with the filename as the company name...
